In this project we'll explore the batch learning, stochastic gradient descent (SGD) and mini batch algorithms for a linear regression problem

In [73]:
import numpy as np
from numpy.random import rand, randn, randint,seed, permutation
from numpy.linalg import inv
from sklearn.linear_model import SGDRegressor
from __future__ import division, print_function
from time import time


Let's start of by generating some pseudo data. 

In [74]:
seed(555)
X = 2 * rand(100,1)
y = 4 + 3 * X + rand(100,1)

Next we create the design matrix

In [75]:
X_b = np.c_[np.ones((100,1)),X]

The closed solution can easily be calculated by using the following formula. Alas, in the case of thousand of variables, it may not be optimal. That is where the SGD algoritm comes in handy

In [4]:
start = time()
theta_best = inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
print('Time to complete: {}'.format(time()-start))  

Time to complete: 0.140156984329


In [5]:
theta_best

array([[ 4.57407633],
       [ 2.96585269]])

The intercept is estimated to be 4.57 and the coefficient 2.97. This is close to the actual parameter values. Nexxt we'll make use of the batch gradient descent algorithm

# Batch gradient descent

We'll start with a 1000 iterations and a learning rate equal to 0.1

In [6]:
seed(355)
eta = 0.1
n_iterations = 1000
m = 100
theta = randn(2,1)

In [7]:
start = time()
for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients
print('Time to complete: {}'.format(time()-start))    

Time to complete: 0.010883808136


In [8]:
theta

array([[ 4.57407633],
       [ 2.96585269]])

This is exaclty the same vlaues we got from the closed solution. We can also create a method to make changing the number of iterations and learning rate easier

In [9]:
class Batch_learning():
    
    def predict(self,X):
        return X.dot(self.theta)
    
    def fit(self,X,Y,eta=0.1,n_iterations=1000):
        
        X_b = np.c_[np.ones((X.shape[0],1)),X]
        self.theta = randn(X_b.shape[1],1)
        m = X_b.shape[0]
        
        for iteration in range(n_iterations):
            y_hat = self.predict(X_b)
            gradients = 2/m * X_b.T.dot(y_hat - y)
            self.theta = self.theta - eta * gradients
        
        
        


In [10]:
batch_learn_mod = Batch_learning()

In [11]:
batch_learn_mod.fit(X,y)

In [12]:
batch_learn_mod.theta

array([[ 4.57407633],
       [ 2.96585269]])

# Stochastic gradient descent

For the stochastic gradient descent algorithm, we'll start with a relatively high learning rate and then decrease it per epoch. The number of epochs will be set equal to 100

In [58]:
class SGD():
    
    def learning_schedule(self,iteration):
        return self.learning_rate * (1 / (1+ self.decay * iteration))
    
    def predict(self,X):
        return X.dot(self.theta)
    
    def fit(self,X,Y,eta0=0.1,n_iterations=50):
        
        self.iterations = n_iterations
        self.decay = eta0 / n_iterations
        self.learning_rate = eta0
        
        X_b = np.c_[np.ones((X.shape[0],1)),X]
        self.theta = randn(X_b.shape[1],1)
        m = X_b.shape[0]
        
        for iteration in range(n_iterations):
            
            self.learning_rate = self.learning_schedule(iteration)
            
            for i in range(m):
                random_index = randint(m)
                xi = X_b[random_index:random_index+1]
                yi = y[random_index:random_index+1]
                y_hat = self.predict(xi)   
                gradients = 2 * xi.T.dot(y_hat - yi)
                self.theta = self.theta - self.learning_rate * gradients


In [61]:
SGD_mod = SGD()
SGD_mod.fit(X,y,eta0=0.1,n_iterations=100)

In [62]:
SGD_mod.theta

array([[ 4.57515336],
       [ 2.97866398]])

This is very close to the closed solution. We can of course also just use sklearn's SGDRegressor method

In [54]:
sgd_reg = SGDRegressor(max_iter = 50, penalty=None, eta0=0.1)
sgd_reg.fit(X,y.ravel())

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.1,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=50, n_iter=None, penalty=None,
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [55]:
sgd_reg.intercept_, sgd_reg.coef_

(array([ 4.56618965]), array([ 2.95393607]))

# Mini-batch

Lastly we'll have a look at the mini-batch algorithm. We'll make use of batch sizes of 20, start with a relatively high learning rate of 0.2 and have 50 iterations

In [ ]:
n_epochs = 50
batch_size = 20

In [ ]:
seed(5445)
theta = randn(2,1)
t = 0
t0, t1 = 200, 1000 #learning schedule hyperparameter

for epoch in range(n_epochs):
    shuffled_indices = permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    
    for i in range(0,m,batch_size):
        t +=1
        xi = X_b[i:i + batch_size]
        yi = y[i:i + batch_size]
        gradients = 2/batch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients

In [71]:
class mini_batch():
    
    def learning_schedule(self,iteration):
        return self.learning_rate * (1 / (1+ self.decay * iteration))
    
    def predict(self,X):
        return X.dot(self.theta)
    
    def fit(self,X,Y,eta0=0.1,n_iterations=50,batch_size=20):
        
        self.iterations = n_iterations
        self.decay = eta0 / n_iterations
        self.learning_rate = eta0
        
        X_b = np.c_[np.ones((X.shape[0],1)),X]
        self.theta = randn(X_b.shape[1],1)
        m = X_b.shape[0]
        
        for iteration in range(n_iterations):
            
            shuffled_indices = permutation(m)
            X_b_shuffled = X_b[shuffled_indices]
            y_shuffled = y[shuffled_indices]
            
            self.learning_rate = self.learning_schedule(iteration)
            
            for i in range(0,m,batch_size):
    
                xi = X_b[i:i + batch_size]
                yi = y[i:i + batch_size]
                y_hat = self.predict(xi)   
                gradients = 2/batch_size * xi.T.dot(y_hat - yi)
                self.theta = self.theta - self.learning_rate * gradients


In [76]:
SGD_mod = mini_batch()
SGD_mod.fit(X,y,eta0=0.2)

In [77]:
SGD_mod.theta

array([[ 4.57590897],
       [ 2.96586903]])